In [1]:
#! /usr/bin/env python3
%matplotlib widget
import rospy
import sys
import select
import actionlib
import actionlib.msg
import assignment_2_2022.msg

from assignment_2_2022.srv import target_srv
from nav_msgs.msg import Odometry
from assignment_2_2022.msg import PoseVelocity
from geometry_msgs.msg import Point, Pose, Twist 
from threading import Thread

import matplotlib.pyplot as plt
import tf
from nav_msgs.msg import Odometry
from tf.transformations import quaternion_matrix
import numpy as np
from matplotlib.animation import FuncAnimation  
from my_module import Visualiser

In [ ]:
class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro')
        self.x_data, self.y_data = [] , []

    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.ln

    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)

    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln
    
vis = Visualiser()



#rospy.init_node('odom_vis_node')
#vis = Visualiser()
sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
plt.show(block=True)

In [ ]:
def call_srv():

    rospy.wait_for_service ('srv')
    srv = rospy.ServiceProxy ('srv', target_srv)
    resp = srv()
    print("Number of goals cancelled", resp.cancelled)
    print("Number of goals reached", resp.reached)
  

def pub_data (msg): #function to publish information on position

    global pub

    PoseVelocity_output = PoseVelocity() #custom msg
    position = msg.pose.pose.position #get position
    velocity = msg.twist.twist.linear #get twist

    #parameters
    PoseVelocity_output_px = position.x
    PoseVelocity_output_py = position.y
    PoseVelocity_output_vx = velocity.x
    PoseVelocity_output_vy = velocity.y

    pub.publish(PoseVelocity_output) #publish message


def action_client():

    action_client = actionlib.SimpleActionClient("/reaching_goal", assignment_2_2022.msg.PlanningAction)

    action_client.wait_for_server()

    x=0
    y=0
        
    while not rospy.is_shutdown():
        print (" Enter the coordinates x and y")
        x=float(input("Enter a goal x: "))
        y=float(input("Enter a goal y: "))
        
        #target position
        goal = assignment_2_2022.msg.PlanningGoal()
        goal.target_pose.pose.position.x = x
        goal.target_pose.pose.position.y = y

        target_x = x
        target_y = y

        action_client.send_goal(goal)

    
def cancel_target(self):

    print("press x to cancel the target")
    input=select.select ([sys.stdin],  [], [], 3) [0] #give user 3 seconds to cancel goal

    if input:
        i = sys.stdin.readline().rstrip()
        if(i=="x"):
            self.action_client.cancel_goal()
            print("The goal has been cancelled")
            self.target_x = None
            self.target_y = None

def main():

    rospy.init_node("action_client")
    global pub

    #publisher for custom message
    pub = rospy.Publisher("/PoseVelocity", PoseVelocity, queue_size=10)


    #subscriber for /odom
    odom_sub = rospy.Subscriber("/odom", Odometry, pub_data)
    
    action_client()
    
if __name__ == '__main__':
        main()
        

In [ ]:
#widgets to sends and cancel the goals
#shows number of reached and cancelled goals
#visualise